Yelp API Calls

Laura Dimaria

7/25/22

In [51]:
import pandas as pd
import json
from yelpapi import YelpAPI

In [52]:
with open('/Users/lauradimaria/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [53]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [54]:
french = yelp_api.search_query(location='Philadelphia, PA',
                               term='French')

type(french)
french.keys()

dict_keys(['businesses', 'total', 'region'])

In [55]:
french_df = pd.DataFrame(french['businesses'])
french_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,4_-IcMpkF_sBRHomWZHNzA,bistrot-la-minette-philadelphia-2,Bistrot La Minette,https://s3-media3.fl.yelpcdn.com/bphoto/JVcztQ...,False,https://www.yelp.com/biz/bistrot-la-minette-ph...,625,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 39.9414622968223, 'longitude': -7...",[delivery],$$$,"{'address1': '623 S 6th St', 'address2': '', '...",+12159258000,(215) 925-8000,2109.506435
1,j-qtdD55OLfSqfsWuQTDJg,parc-philadelphia-2,Parc,https://s3-media3.fl.yelpcdn.com/bphoto/0vGzdK...,False,https://www.yelp.com/biz/parc-philadelphia-2?a...,2893,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.0,"{'latitude': 39.9491724, 'longitude': -75.1707...","[pickup, delivery]",$$$,"{'address1': '227 S 18th St', 'address2': '', ...",+12155452262,(215) 545-2262,514.179687
2,KTgZXj6xh8aN_tLfI-YZ1Q,bar-poulet-philadelphia,Bar Poulet,https://s3-media3.fl.yelpcdn.com/bphoto/lqt4KI...,False,https://www.yelp.com/biz/bar-poulet-philadelph...,86,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 39.95079, 'longitude': -75.17399}",[],NaN,"{'address1': '2005 Walnut St', 'address2': '',...",+12155578277,(215) 557-8277,407.607091
3,Oz7c0QA1C18Cb6tTdoehwQ,gabi-philadelphia,Gabi,https://s3-media4.fl.yelpcdn.com/bphoto/2Zpafl...,False,https://www.yelp.com/biz/gabi-philadelphia?adj...,129,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 39.95925, 'longitude': -75.16151}",[delivery],$$,"{'address1': '339 N Broad St', 'address2': '',...",,,1008.936953
4,p-MSh95si_sDdih94gyKuw,forsythia-philadelphia,Forsythia,https://s3-media3.fl.yelpcdn.com/bphoto/3jnQZf...,False,https://www.yelp.com/biz/forsythia-philadelphi...,136,"[{'alias': 'modern_european', 'title': 'Modern...",4.5,"{'latitude': 39.9487067926056, 'longitude': -7...",[delivery],$$$,"{'address1': '233 Chestnut St', 'address2': ''...",+12156449395,(215) 644-9395,2271.260618


In [56]:
french['total']

132

132 results would need 7 pages.

In [57]:
from tqdm.notebook import tqdm_notebook

In [58]:
import math, time, os

In [59]:
french_json_progress = f"Data/results_in_progress_french.json"
french_json_progress

'Data/results_in_progress_french.json'

In [60]:
file_exists = os.path.isfile(french_json_progress)
if file_exists == False:
    folder = os.path.dirname(french_json_progress)
    if len(folder)>0:
        os.makedirs(folder,exist_ok=True)
        

    print(f"[i] {french_json_progress} not found. Saving empty list to file.")
    
    with open(french_json_progress,'w') as f:
        json.dump([],f)  

else:
    print(f"[i] {french_json_progress} already exists.")

[i] Data/results_in_progress_french.json already exists.


In [61]:
with open(french_json_progress,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 132 previous results found.


In [62]:
results = yelp_api.search_query(location="Philadelphia, PA",
                                term="French",
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [63]:
total_results = results['total']
results_per_page = len(results['businesses'])

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

ZeroDivisionError: division by zero

In [64]:
previous_results.extend(results['businesses'])  
with open(french_json_progress,'w') as f:
     json.dump(previous_results,f)

In [65]:
for i in tqdm_notebook(range(n_pages)):
    time.sleep(.2) 

  0%|          | 0/7 [00:00<?, ?it/s]

In [66]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)

    with open(french_json_progress, 'r') as f:
        previous_results = json.load(f)

    n_results = len(previous_results)

    results = yelp_api.search_query(location='Philadelphia, PA',
                                    term='French', 
                                    offset=n_results)
    

    previous_results.extend(results['businesses'])
    

    with open(french_json_progress,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/7 [00:00<?, ?it/s]

In [67]:
final_french_df = pd.read_json(french_json_progress)
display(final_french_df.head(), final_french_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,4_-IcMpkF_sBRHomWZHNzA,bistrot-la-minette-philadelphia-2,Bistrot La Minette,https://s3-media3.fl.yelpcdn.com/bphoto/JVcztQ...,False,https://www.yelp.com/biz/bistrot-la-minette-ph...,625,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 39.9414622968223, 'longitude': -7...",[delivery],$$$,"{'address1': '623 S 6th St', 'address2': '', '...",+12159258000,(215) 925-8000,2109.506435
1,j-qtdD55OLfSqfsWuQTDJg,parc-philadelphia-2,Parc,https://s3-media3.fl.yelpcdn.com/bphoto/0vGzdK...,False,https://www.yelp.com/biz/parc-philadelphia-2?a...,2893,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.0,"{'latitude': 39.9491724, 'longitude': -75.1707...","[pickup, delivery]",$$$,"{'address1': '227 S 18th St', 'address2': '', ...",+12155452262,(215) 545-2262,514.179687
2,KTgZXj6xh8aN_tLfI-YZ1Q,bar-poulet-philadelphia,Bar Poulet,https://s3-media3.fl.yelpcdn.com/bphoto/lqt4KI...,False,https://www.yelp.com/biz/bar-poulet-philadelph...,86,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 39.95079, 'longitude': -75.17399}",[],NaN,"{'address1': '2005 Walnut St', 'address2': '',...",+12155578277,(215) 557-8277,407.607091
3,Oz7c0QA1C18Cb6tTdoehwQ,gabi-philadelphia,Gabi,https://s3-media4.fl.yelpcdn.com/bphoto/2Zpafl...,False,https://www.yelp.com/biz/gabi-philadelphia?adj...,129,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 39.95925, 'longitude': -75.16151}",[delivery],$$,"{'address1': '339 N Broad St', 'address2': '',...",,,1008.936953
4,p-MSh95si_sDdih94gyKuw,forsythia-philadelphia,Forsythia,https://s3-media3.fl.yelpcdn.com/bphoto/3jnQZf...,False,https://www.yelp.com/biz/forsythia-philadelphi...,136,"[{'alias': 'modern_european', 'title': 'Modern...",4.5,"{'latitude': 39.9487067926056, 'longitude': -7...",[delivery],$$$,"{'address1': '233 Chestnut St', 'address2': ''...",+12156449395,(215) 644-9395,2271.260618


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
127,ArNDRug3p5zngNHiLivnFA,cibo-bistro-wine-bar-philadelphia,CIBO Bistro Wine Bar,https://s3-media3.fl.yelpcdn.com/bphoto/_7yKMN...,False,https://www.yelp.com/biz/cibo-bistro-wine-bar-...,135,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.0,"{'latitude': 39.875016, 'longitude': -75.243913}",[],$$,"{'address1': '8000 Essington Ave', 'address2':...",+12676845003,(267) 684-5003,10731.427037
128,19fHbf0qzihtEtXQZ2ntww,walters-steakhouse-wilmington,Walter's Steakhouse,https://s3-media2.fl.yelpcdn.com/bphoto/Rotj6k...,False,https://www.yelp.com/biz/walters-steakhouse-wi...,92,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 39.75229, 'longitude': -75.57062}",[],$$$,"{'address1': '802 N Union St', 'address2': '',...",+13026526780,(302) 652-6780,40798.199868
129,G0Fhi9lh1sSZSSPuXpe6Ug,harvest-seasonal-grill-newtown-newtown,Harvest Seasonal Grill - Newtown,https://s3-media2.fl.yelpcdn.com/bphoto/3KoRJS...,False,https://www.yelp.com/biz/harvest-seasonal-gril...,283,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 40.2357309, 'longitude': -74.9394...",[delivery],$$,"{'address1': '2865 S Eagle Rd', 'address2': No...",+12159448469,(215) 944-8469,37033.065998
130,rK4MQ-vybY5KXI73o-0mlw,jakes-and-coopers-wine-bar-and-wine-shop-phila...,Jake's & Cooper's Wine Bar & Wine Shop,https://s3-media3.fl.yelpcdn.com/bphoto/tD7zPa...,False,https://www.yelp.com/biz/jakes-and-coopers-win...,319,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 40.025878, 'longitude': -75.224113}","[delivery, pickup]",$$,"{'address1': '4365 Main St', 'address2': '', '...",+12154830444,(215) 483-0444,9204.431323
131,8mSY98peCMoPaHcUhfeo9A,seven-star-diner-sewell,Seven Star Diner,https://s3-media2.fl.yelpcdn.com/bphoto/7Fhrkr...,False,https://www.yelp.com/biz/seven-star-diner-sewe...,230,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",2.5,"{'latitude': 39.78461, 'longitude': -75.102223}",[delivery],$$,"{'address1': '1890 Hurfville Rd', 'address2': ...",,,19712.105055


In [68]:
final_french_df.to_csv('Data/final_results_french.csv.gz', compression='gzip',index=False)